## FFF Vision - Hough Transform Method

In [ ]:
#Importing all necessary packages.
import numpy as np
import cv2
print('OpenCV version: '+cv2.__version__)
import pandas as pd
import datetime
import os
import time
import re 

In [ ]:
#used for time calculation.
start = time.perf_counter()
#Reading the folder where the videos are saved.
SRC_FOLDER = r"D:\Pensulvaniya state project\FFF Vision - OpenCV [External]\Vidseos\\"
#reading the vieo_Time_Stamp from device.
df_vidTimes = pd.read_excel(SRC_FOLDER + "Video_Timestamps_1.xlsx")
df_vidTimes.drop(df_vidTimes.columns[0],axis=1,inplace=True)

In [ ]:
#Function for image perspective correction.
def perspCorrection(img,pt1,pt2,pt3,pt4,scale_width,scale_height):
    
    input_pts = np.float32([pt1,pt2,pt3,pt4])
    output_pts = np.float32([[0,0],[scale_width-1,0],[0,scale_height-1],[scale_width-1,scale_height-1]])
    M = cv2.getPerspectiveTransform(input_pts,output_pts)
    #saves the perspective corrected image to a varibale.
    imgPersp = cv2.warpPerspective(img,M,(scale_width, scale_height)) 
    #grayscale conversion.
    imgGrayPersp = cv2.cvtColor(imgPersp, cv2.COLOR_BGR2GRAY)            
    
    return [imgPersp,imgGrayPersp]


In [ ]:
# function for hough transform and width calculation
def hough(gray_image,mm_per_pixel):
    
    # Canny edge detection
    edges = cv2.Canny(gray_image, 50, 150, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi/180, 150)
    for r_theta in lines:
        arr = np.array(r_theta[0], dtype=np.float64)
        r, theta = arr
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*r
        y0 = b*r
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(edges, (x1, y1), (x2, y2), 125, 2)
    
    # Extrusion width calculation
    indices = np.where(edges == 125)
    y_indices = indices[1]
    avg = np.mean(indices[1])
    LEdges = y_indices[np.where(y_indices < avg)]
    REdges = y_indices[np.where(y_indices > avg)]
    width_px = np.abs(np.mean(REdges)-np.mean(LEdges))
    width_in_mm = mm_per_pixel * width_px
    
    text = str(width_in_mm) + ' mm'
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50,50)
    fontScale = 1
    color = (255, 0, 0)
    thickness = 2
    img_color = cv2.putText(imgGrayPersp, text, org, font,fontScale, color, thickness)

    
    return [width_px, width_in_mm, img_color, edges]

In [ ]:
# Remove # for below code to save the path of folder in which images for analysis is to be saved
#imganalysis_path = SRC_FOLDER + "Results/Run-2"

#function for storing images for analysis, need to call this function from recursive function.

def image_for_analysis(imganalysis_path,layer,speed,frameCount,edges,imgcolor):
    
    #for storing perspective corrected grayscale images.
    cv2.imwrite(imganalysis_path + '/' + str(layers) + "/" + str(speeds) + "/Gray/" +  "pCorr" + str(frameCount) + ".jpg", imgGrayPersp)
    
    #for storing thersholded images.                       
    cv2.imwrite(imganalysis_path +  '/' + str(layers) + "/" + str(speeds) + "/Thresholded Images/" + "thresh" + str(frameCount) + ".jpg", thersholded_image)
    
    #for storing output images of  median blur function.                     
    cv2.imwrite(imganalysis_path +  '/' + str(layers) + "/" + str(speeds) + "/median blur/" + "medianblur" + str(frameCount) + ".jpg", medianblr)
    
    #for storing output images of  morphological operations(dilation and erosion).                     
    cv2.imwrite(imganalysis_path +  '/' + str(layers) + "/" + str(speeds) + "/Erosion and Dilation/" + "morphOps" + str(frameCount) + ".jpg", cloperation)
    
    #for storing canny images with detected edges.                         
    cv2.imwrite(imganalysis_path +  '/' + str(layer) + "/" + str(speed) + "/Canny/" + "Canny" + str(frameCount) + ".jpg", edges)
    
    #for storing the ROI selected images.                        
    #cv2.imwrite(imganalysis_path +  '/' + str(layers) + "/" + str(speeds) + "/ROI/" + "ROI" + str(frameCount) + ".jpg", bottom_image)    
    
    #for storing the images with labeled width measures.
    cv2.imwrite(imganalysis_path +  '/' + str(layer) + "/" + str(speed) + "/Vision Measurements/" + "wImg" + str(frameCount) + ".jpg", imgcolor)
    
    return None
#This function returns nothing since it only save images to respective folders in system.
#There should be image folders as mentioned in the save_path

In [ ]:
#Main program
num_layers = 20
max_speed = 50
layers = list(range(5,num_layers+1))
speeds = list(range(10,max_speed+10,10))
frame_skip_start = [32,20,11,15,13] 

vidCount = 0
img_debug = False 

w_result_columns=['Layer','vR','Frame','ActualTimestamp','Edge Distance in Pixel','w_Vision']
frame_summary_columns = ['Layer','vR','Start_TS','End_TS','Total_Frames','Per_Frames_Skipped','Skipped_Frames']
lst = []
lst_skip_frames = []
lst_frame_summary = []

pt1 = [192.30,343.00]  
pt2 = [1079.0,379.80]  
pt3 = [153.50,571.90] 
pt4 = [1107.10,611.70] 

scale_width = round(11.7348*200) 
scale_height = round(6.35*200)   

bStart = [655,925]
bEnd = [1300,1270]
fsize =9
threshold_1 =30
threshold_2 = 80
mm_per_pixel = 0.004992138364779874


for l in range(len(layers)):
    for v in range(len(speeds)):
        
        lst = []
        lst_skip_frames = []
                
        vidName = 'vid_l'+str(layers[l])+'_vR_'+str(speeds[v])+'.avi'
        print('Processing video: ' + vidName)
        
        idx = df_vidTimes.index[(df_vidTimes.Layer==layers[l]) & (df_vidTimes.Speed==speeds[v])].to_list()[0]
        start_TS = df_vidTimes.Start_Timestamp[idx]
        end_TS = df_vidTimes.End_Timestamp[idx]
        
        print('video: {0} starts at {1} and ends at {2}'.format(vidName,start_TS,end_TS))
        
        srcVideo = SRC_FOLDER + vidName       
        cap = cv2.VideoCapture(srcVideo)
        numFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        print('video {0} has {1} frames'.format(vidName,numFrames))

        if (cap.isOpened() == False):
            print("Error reading video file. Exiting ...")
            exit(0)
     
            
            
        frameCount = 0

        while(cap.isOpened()):
    
            frame_exists, frame = cap.read()
               
               
            if frame_exists:
                
                frameCount = frameCount + 1
                

                if(frameCount >= frame_skip_start[v] and frameCount <= numFrames - 5): 
                    
                    try:
                                               

                        # call function - correct perspective transform
                        [imgPersp,imgGrayPersp] = perspCorrection(frame,pt1,pt2,pt3,pt4,scale_width,scale_height)
                        
                        

                         #Hough Transformation
                         # Extrusion width measurement
                        [width_px, width_in_mm, imgcolor, edges] = hough(imgGrayPersp,mm_per_pixel) 
                        #cv2.imwrite(patha + str(layers[l]) + str(speeds[v]) + str(frameCount) + ".jpg", imgcolor)
      
                        
                        #Remove Comment line for below code , if images needs to be saved for analysing.
                        #image_for_analysis(imganalysis_path,layers[l],speeds[v],frameCount,edges,imgcolor)

                        # Calculate actual timestamp based on excel timestamps and frame number
                        act_TS = start_TS+frameCount*(end_TS-start_TS)/numFrames

                        # Store results in dataframe   
                        lst.append([layers[l],speeds[v],frameCount,act_TS,width_px, width_in_mm])
                        

                        
                    except ValueError as e:
                        
                        print('Unable to sucessfully process frame {0}, skipping . . .'.format(frameCount))
                        print(e)
                        # Calculate actual timestamp based on excel timestamps and frame number
                        act_TS = start_TS+frameCount*(end_TS-start_TS)/numFrames
                        # Store results in dataframe   
                        lst.append([layers[l],speeds[v],frameCount,act_TS,np.nan,np.nan])
                        lst_skip_frames.append([frameCount])

                    except UnboundLocalError as u:
                        print('Unable to sucessfully process frame {0}, skipping . . .'.format(frameCount))
                        print(u)
                        # Calculate actual timestamp based on excel timestamps and frame number
                        act_TS = start_TS+frameCount*(end_TS-start_TS)/numFrames
                        # Store results in dataframe   
                        lst.append([layers[l],speeds[v],frameCount,act_TS,np.nan,np.nan])
                        lst_skip_frames.append([frameCount])
                        
                    except TypeError as t:
                        print('Unable to sucessfully process frame {0}, skipping . . .'.format(frameCount))
                        print("Necessary lines are not detected in the frame") 
                        # Calculate actual timestamp based on excel timestamps and frame number
                        act_TS = start_TS+frameCount*(end_TS-start_TS)/numFrames
                        # Store results in dataframe   
                        lst.append([layers[l],speeds[v],frameCount,act_TS,np.nan,np.nan])
                        lst_skip_frames.append([frameCount])

            else:
                
                break       
                   
        cap.release()
        cv2.destroyAllWindows()
        
        print('Finished processing video: {0}'.format(vidName))
        print('')
        print('')
        vidCount = vidCount + 1
    
        results = pd.DataFrame(lst,columns=w_result_columns)
        # Save results to csv file
        path = SRC_FOLDER + "Results/Run-2/" + 'l' + str(layers[l])+'_vR'+str(speeds[v])+"results1.csv"
        results.to_csv(path)
        lst_frame_summary.append([layers[l],speeds[v],start_TS,end_TS,numFrames,len(lst_skip_frames)/numFrames,lst_skip_frames])


frame_summary_results = pd.DataFrame(lst_frame_summary,columns=frame_summary_columns)

# Some more cleanup and data addition
frame_summary_results["Video_Duration"] = frame_summary_results["End_TS"] - frame_summary_results["Start_TS"] 
frame_summary_results["Video_Duration"] = [x.total_seconds() for x in frame_summary_results["Video_Duration"]]
frame_summary_results["FPS"] = frame_summary_results["Total_Frames"]/frame_summary_results["Video_Duration"]
frame_summary_results["Total_Frames_Skipped"] = [len(x) for x in frame_summary_results["Skipped_Frames"]]

# Re-oder columns
frame_summary_results = frame_summary_results[["Layer","vR","Start_TS","End_TS","Video_Duration","Total_Frames","FPS","Total_Frames_Skipped","Per_Frames_Skipped","Skipped_Frames"]]

#Writing to CSV
path_summary = SRC_FOLDER + "Results/Run-2/"+ 'video_processing_summary.csv'
frame_summary_results.to_csv(path_summary)
    
print('Processing of all videos completed successfully! Summary results saved at {0}'.format(summary_path))
end = time.perf_counter() - start
print('{:.6f}s for the calculation'.format(end))